In [1]:
import os

from josie import JOSIE

In [2]:
data_path = os.path.join("..", "data", "open_data", "nyc")

fmt = "parquet"

tables_path = os.path.join(data_path, "tables", fmt)
metadata_path = os.path.join(data_path, "metadata", "metadata.json")
output_terms_path = os.path.join(data_path, "tmp", "flat_terms.txt")
output_terms_data_path = os.path.join(data_path, "tmp", "flat_terms_data.txt")
output_terms_queries_path = os.path.join(data_path, "tmp", "flat_terms_queries.txt")

In [3]:
with open(output_terms_path) as infile:
    lines = infile.readlines()
    with open(output_terms_data_path, 'w') as outfile:
        for i, line in enumerate(lines[:]):
            outfile.write(f"{i} {' '.join(line.split(' ')[1:])}")
    
    queries_id = list(enumerate(lines))[:-20]

In [4]:
db_config = {
    "host": "127.0.0.1",
    "port": 5442,
    "database": "josie",
    "drivername": "postgresql",
    "username": "nanni",
    "password": ""
}

spark_config = {
    "spark.app.name":               "JOSIE Data Preparation",
    "spark.master":                 "local[9]",
    "spark.executor.memory":        "100g",
    "spark.driver.memory":          "20g",
    "spark.local.dir":              "/data3/nanni/.spark_tmp",
    "spark.driver.maxResultSize":   "12g",
    "spark.jars.packages":          "org.postgresql:postgresql:42.7.4",
    "spark.driver.extraClassPath":  "/home/nanni/.ivy2/jars/org.postgresql_postgresql-42.7.3.jar"
}

js = JOSIE(output_terms_data_path, db_config, spark_config=spark_config)

In [12]:
js.data_preparation()

Creating integer sets and inverted index tables...


:: loading settings :: url = jar:file:/data4/nanni/anaconda3/envs/datascience3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/nanni/.ivy2.5.2/cache
The jars for the packages stored in: /home/nanni/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-becfebd3-9250-440b-bfaf-70ec7d284dec;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.4 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 140ms :: artifacts dl 5ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnld

Total posting lists: 9927
Total integer sets: 925


Integer sets saved


Posting lists saved
Spark session stopped


In [6]:
js.db.load_tables()

queries = [
    { 
        'id': query_id,
        'tokens': js.db.get_set_tokens(query_id)
    }
    for query_id, line in queries_id
    if len(line.strip().split(' ')[1:]) > 0
][:10]

In [7]:
results = js.query(queries, 5)

Creating token table on disk...
Begin experiment for k=5...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.39it/s]

Finished experiment for k=5 in 0.012 minutes


In [10]:
results[0]

[[0, [(3, 240), (2, 237), (5, 235), (7, 233), (8, 231)]],
 [1, [(9, 148), (16, 140), (4, 139), (27, 138), (25, 128)]],
 [2, [(5, 246), (3, 245), (0, 237), (7, 234), (8, 234)]],
 [3, [(5, 247), (2, 245), (7, 243), (8, 243), (0, 240)]],
 [4, [(9, 211), (25, 195), (16, 193), (14, 187), (27, 185)]],
 [5, [(3, 247), (2, 246), (0, 235), (7, 234), (8, 234)]],
 [6, [(4, 182), (16, 179), (14, 171), (27, 171), (25, 163)]],
 [7, [(8, 250), (3, 243), (5, 234), (2, 234), (0, 233)]],
 [8, [(7, 250), (3, 243), (5, 234), (2, 234), (0, 231)]],
 [9, [(4, 211), (14, 190), (25, 184), (16, 178), (27, 170)]]]